In [1]:
import itertools
import numpy as np
from typing import Dict, List, Tuple
import logging
from experiment1_solution_quality import DynamicProgramming, DPState

In [6]:
class DPValidator:
    """Validation framework for the Dynamic Programming solution.
    
    This class implements various tests to verify the correctness of the DP solution
    by checking mathematical properties that must hold true for the optimal policy.
    """
    
    def __init__(self, dp_solver: DynamicProgramming):
        """Initialize validator with a DP solver instance."""
        self.dp = dp_solver
        self.logger = logging.getLogger(__name__)
    
    def validate_solution(self) -> Dict[str, bool]:
        """Run all validation checks on the DP solution.
        
        Returns:
            Dictionary mapping test names to boolean results
        """
        self.logger.info("Starting validation of DP solution")
        results = {}
        
        # Run all validation checks
        results['monotonicity'] = self.validate_monotonicity()
        results['boundary_conditions'] = self.validate_boundary_conditions()
        results['concavity'] = self.validate_concavity()
        results['time_consistency'] = self.validate_time_consistency()
        results['capacity_constraints'] = self.validate_capacity_constraints()
        results['optimal_substructure'] = self.validate_optimal_substructure()
        
        # Log overall results
        passed = all(results.values())
        self.logger.info(f"Validation {'passed' if passed else 'failed'}")
        for test, result in results.items():
            self.logger.info(f"{test}: {'passed' if result else 'failed'}")
            
        return results
    
    def validate_monotonicity(self) -> bool:
        """Verify that value function is monotonically increasing in capacity.
        
        The value function should be non-decreasing in capacity: having more rooms
        available cannot decrease the optimal expected revenue.
        """
        self.logger.info("Checking value function monotonicity")
        
        for t in range(1, self.dp.T + 1):
            capacities = self.dp._generate_capacity_vectors()
            for cap1 in capacities:
                state1 = DPState(capacity=cap1, time=t)
                value1 = self.dp.value_function.get(state1)
                if value1 is None:
                    continue
                    
                for cap2 in capacities:
                    # Skip if cap2 is not componentwise greater than cap1
                    if not np.all(cap2 >= cap1):
                        continue
                        
                    state2 = DPState(capacity=cap2, time=t)
                    value2 = self.dp.value_function.get(state2)
                    if value2 is None:
                        continue
                        
                    if value2 < value1:
                        self.logger.error(
                            f"Monotonicity violated at t={t}:\n"
                            f"V({cap1}, {t}) = {value1} > V({cap2}, {t}) = {value2}"
                        )
                        return False
        
        return True
    
    def validate_boundary_conditions(self) -> bool:
        """Verify that boundary conditions are satisfied.
        
        Checks:
        1. V(x, T+1) = 0 for all capacity vectors x
        2. V(0, t) = 0 for all time periods t
        """
        self.logger.info("Checking boundary conditions")
        
        # Check terminal conditions
        terminal_time = self.dp.T + 1
        for capacity in self.dp._generate_capacity_vectors():
            state = DPState(capacity=capacity, time=terminal_time)
            if self.dp.value_function.get(state, 0.0) != 0.0:
                self.logger.error(
                    f"Terminal condition violated: V({capacity}, {terminal_time}) ≠ 0"
                )
                return False
        
        # Check zero capacity conditions
        zero_capacity = np.zeros(self.dp.N, dtype=np.int32)
        for t in range(1, self.dp.T + 2):
            state = DPState(capacity=zero_capacity, time=t)
            if self.dp.value_function.get(state, 0.0) != 0.0:
                self.logger.error(
                    f"Zero capacity condition violated: V(0, {t}) ≠ 0"
                )
                return False
        
        return True
    
    def validate_concavity(self) -> bool:
        """Verify that value function is concave in capacity.
        
        The marginal value of additional capacity should be decreasing.
        """
        self.logger.info("Checking value function concavity")
        
        for t in range(1, self.dp.T + 1):
            capacities = self.dp._generate_capacity_vectors()
            for cap in capacities:
                state = DPState(capacity=cap, time=t)
                value = self.dp.value_function.get(state)
                if value is None:
                    continue
                
                # Check each dimension
                for day in range(self.dp.N):
                    if cap[day] >= 2:  # Need at least 2 capacity for second difference
                        cap_minus_1 = cap.copy()
                        cap_minus_1[day] -= 1
                        cap_minus_2 = cap.copy()
                        cap_minus_2[day] -= 2
                        
                        state_minus_1 = DPState(capacity=cap_minus_1, time=t)
                        state_minus_2 = DPState(capacity=cap_minus_2, time=t)
                        
                        value_minus_1 = self.dp.value_function.get(state_minus_1)
                        value_minus_2 = self.dp.value_function.get(state_minus_2)
                        
                        if (value_minus_1 is not None and value_minus_2 is not None):
                            # Check second difference
                            if (value - value_minus_1) < (value_minus_1 - value_minus_2):
                                self.logger.error(
                                    f"Concavity violated at t={t}, day={day}:\n"
                                    f"First difference: {value - value_minus_1}\n"
                                    f"Second difference: {value_minus_1 - value_minus_2}"
                                )
                                return False
        
        return True
    
    def validate_time_consistency(self) -> bool:
        """Verify that the solution exhibits time consistency.
        
        The optimal decision at any state should remain optimal when reached
        through any sequence of previous decisions.
        """
        self.logger.info("Checking time consistency")
        
        # Sample some states and verify their optimal decisions remain consistent
        for t in range(1, self.dp.T):
            capacities = self.dp._generate_capacity_vectors()
            for cap in capacities:
                state = DPState(capacity=cap, time=t)
                if state not in self.dp.value_function:
                    continue
                    
                # Get optimal value directly
                direct_value = self.dp.value_function[state]
                
                # Compute value through one-step lookahead
                recomputed_value = self.dp._compute_value_function(state)
                
                if abs(direct_value - recomputed_value) > 1e-10:
                    self.logger.error(
                        f"Time consistency violated at t={t}:\n"
                        f"Direct value: {direct_value}\n"
                        f"Recomputed value: {recomputed_value}"
                    )
                    return False
        
        return True
    
    def validate_capacity_constraints(self) -> bool:
        """Verify that capacity constraints are never violated.
        
        For any state transition, the remaining capacity should never become negative.
        """
        self.logger.info("Checking capacity constraints")
        
        for t in range(1, self.dp.T + 1):
            capacities = self.dp._generate_capacity_vectors()
            for cap in capacities:
                state = DPState(capacity=cap, time=t)
                if state not in self.dp.value_function:
                    continue
                    
                # Check all possible booking class arrivals
                current_probs = self.dp.arrival_probs[t]
                for (arrival, departure) in current_probs:
                    # Simulate booking acceptance
                    new_cap = cap.copy()
                    for day in range(arrival - 1, departure):
                        new_cap[day] -= 1
                        if new_cap[day] < 0:
                            self.logger.error(
                                f"Capacity constraint violated at t={t}:\n"
                                f"Booking class ({arrival}, {departure}) accepted\n"
                                f"when capacity was {cap}"
                            )
                            return False
        
        return True
    
    def validate_optimal_substructure(self) -> bool:
        """Verify that the solution exhibits optimal substructure.
        
        The optimal solution to a subproblem should be part of the optimal solution
        to the larger problem.
        """
        self.logger.info("Checking optimal substructure")
        
        # Sample some states and verify their subproblem optimality
        for t in range(1, self.dp.T):
            capacities = self.dp._generate_capacity_vectors()
            for cap in capacities:
                state = DPState(capacity=cap, time=t)
                if state not in self.dp.value_function:
                    continue
                    
                # Get optimal value from value function
                optimal_value = self.dp.value_function[state]
                
                # Verify that no other price gives better value
                for price in np.linspace(self.dp.price_min, self.dp.price_max, 3):
                    value = self._compute_subproblem_value(state, price)
                    if value > optimal_value + 1e-10:
                        self.logger.error(
                            f"Optimal substructure violated at t={t}:\n"
                            f"Found better value {value} > {optimal_value}\n"
                            f"using price {price}"
                        )
                        return False
        
        return True
    
    def _compute_subproblem_value(self, state: DPState, price: float) -> float:
        """Helper method to compute value of a subproblem with fixed price."""
        value = 0.0
        current_probs = self.dp.arrival_probs[state.time]
        
        # No arrival probability
        no_arrival_prob = 1.0 - sum(current_probs.values())
        if no_arrival_prob > 0:
            next_state = DPState(
                capacity=state.capacity.copy(),
                time=state.time + 1
            )
            value += no_arrival_prob * self.dp.value_function[next_state]
        
        # For each possible booking class arrival
        for (arrival, departure), arrival_prob in current_probs.items():
            if arrival_prob <= 0:
                continue
                
            # Check capacity availability
            has_capacity = True
            for day in range(arrival - 1, departure):
                if state.capacity[day] < 1:
                    has_capacity = False
                    break
            
            # Compute acceptance probability
            eps = self.dp.price_sensitivity[(arrival, departure)]
            accept_prob = max(0, 1 - eps * price)
            
            if has_capacity and accept_prob > 0:
                # State transition for accepted booking
                next_capacity = state.capacity.copy()
                for day in range(arrival - 1, departure):
                    next_capacity[day] -= 1
                next_state = DPState(
                    capacity=next_capacity,
                    time=state.time + 1
                )
                
                stay_length = departure - arrival + 1
                immediate_revenue = stay_length * price
                value += arrival_prob * accept_prob * (
                    immediate_revenue + self.dp.value_function[next_state]
                )
            
            # Handle rejection case
            reject_prob = 1 - accept_prob
            next_state = DPState(
                capacity=state.capacity.copy(),
                time=state.time + 1
            )
            value += arrival_prob * reject_prob * self.dp.value_function[next_state]
        
        return value

def create_test_instance() -> Tuple[DynamicProgramming, Dict]:
    """Create a small test instance for validation."""
    # Define a minimal problem instance
    T = 2  # booking periods
    N = 3  # service days
    C = 5  # room capacity
    
    # Define booking classes (arrival_day, departure_day)
    booking_classes = [(1,1), (1,2), (1,3), (2, 2), (2, 3), (3,3)]
    
    # Define arrival probabilities (0.15 for all booking classes in each period)
    arrival_probs = {
        t: {bc: 0.15 for bc in booking_classes}
        for t in range(1, T+1)
    }
    
    # Price parameters
    price_min = 50
    price_max = 150
    price_levels = np.linspace(price_min, price_max, 3)  # Discrete price levels for testing
    price_bounds = (price_min, price_max)
    
    # Define price sensitivity parameters
    # Calculate epsilon from uniform distribution parameters
    epsilon = 1/price_max  # = 1/150 ≈ 0.00667
    
    # Define price sensitivity parameters (uniform for all classes)
    price_sensitivity = {bc: epsilon for bc in booking_classes}
    
    # Create DP solver instance
    dp = DynamicProgramming(
        T=T,
        N=N,
        C=C,
        booking_classes=booking_classes,
        arrival_probs=arrival_probs,
        price_sensitivity=price_sensitivity,
        price_bounds=price_bounds
    )
    
    # Return solver and instance details
    return dp, {
        'T': T,
        'N': N,
        'C': C,
        'booking_classes': booking_classes,
        'arrival_probs': arrival_probs,
        'price_sensitivity': price_sensitivity,
        'price_bounds': price_bounds,
        'price_levels': price_levels
    }

def print_optimal_solution(dp: DynamicProgramming, value_function: Dict[DPState, float]):
    """Print optimal prices and value function for each state and period."""
    print("\nOptimal Solution Details:")
    print("=" * 80)
    
    for t in range(1, dp.T + 1):
        print(f"\nBooking Period {t}")
        print("-" * 40)
        
        # Sort states by capacity vector for readable output
        states = [s for s in value_function.keys() if s.time == t]
        states.sort(key=lambda s: tuple(s.capacity))
        
        print(f"{'Capacity Vector':<20} {'Value Function':<15} {'Optimal Price':<15}")
        print("-" * 50)
        
        for state in states:
            # Skip terminal states
            if state.time > dp.T:
                continue
                
            value = value_function[state]
            # Compute optimal price for this state
            # optimal_price = dp._compute_optimal_price(state)
            
            capacity_str = str(tuple(state.capacity))
            # print(f"{capacity_str:<20} {value:<15.2f} {optimal_price:<15.2f}")
            print(f"{capacity_str:<20} {value:<15.2f}")

def run_validation():
    """Run validation tests on a test instance."""
    # Create test instance
    dp, instance_details = create_test_instance()
    
    # Solve DP
    optimal_value, value_function = dp.solve()
    
    # Create validator
    validator = DPValidator(dp)
    
    # Run validation
    results = validator.validate_solution()
    
    return results, optimal_value, instance_details

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    
    # Create and solve test instance
    dp, instance = create_test_instance()
    optimal_value, value_function = dp.solve()
    
    # Run validation
    validator = DPValidator(dp)
    validation_results = validator.validate_solution()
    
    # Print results
    print("\nValidation Results:")
    print("=" * 80)
    for test, passed in validation_results.items():
        print(f"{test}: {'Passed' if passed else 'Failed'}")
    
    print("\nInstance Details:")
    print("=" * 80)
    print(f"Time Periods (T): {instance['T']}")
    print(f"Service Days (N): {instance['N']}")
    print(f"Room Capacity (C): {instance['C']}")
    print(f"Price Range: {instance['price_bounds']}")
    print(f"Price Levels: {instance['price_levels']}")
    print(f"Price Sensitivity (ε): {list(instance['price_sensitivity'].values())[0]:.6f}")
    
    # Print optimal solution details
    print_optimal_solution(dp, value_function)

INFO:experiment1_solution_quality:Initialized DP solver with T=2, N=3, C=5
INFO:experiment1_solution_quality:Starting DP solution
INFO:experiment1_solution_quality:Processing time period 2
INFO:experiment1_solution_quality:Processing time period 1
INFO:__main__:Starting validation of DP solution
INFO:__main__:Checking value function monotonicity
INFO:__main__:Checking boundary conditions
INFO:__main__:Checking value function concavity
ERROR:__main__:Concavity violated at t=1, day=2:
First difference: 0.3357359603566543
Second difference: 9.002379960730647
INFO:__main__:Checking time consistency
INFO:__main__:Checking capacity constraints
ERROR:__main__:Capacity constraint violated at t=1:
Booking class (1, 1) accepted
when capacity was [0 0 0]
INFO:__main__:Checking optimal substructure
INFO:__main__:Validation failed
INFO:__main__:monotonicity: passed
INFO:__main__:boundary_conditions: passed
INFO:__main__:concavity: failed
INFO:__main__:time_consistency: passed
INFO:__main__:capacity


Validation Results:
monotonicity: Passed
boundary_conditions: Passed
concavity: Failed
time_consistency: Passed
capacity_constraints: Failed
optimal_substructure: Passed

Instance Details:
Time Periods (T): 2
Service Days (N): 3
Room Capacity (C): 5
Price Range: (50, 150)
Price Levels: [ 50. 100. 150.]
Price Sensitivity (ε): 0.006667

Optimal Solution Details:

Booking Period 1
----------------------------------------
Capacity Vector      Value Function  Optimal Price  
--------------------------------------------------
(0, 0, 0)            0.00           
(0, 0, 1)            9.00           
(0, 0, 2)            9.34           
(0, 0, 3)            9.34           
(0, 0, 4)            9.34           
(0, 0, 5)            9.34           
(0, 1, 0)            9.00           
(0, 1, 1)            36.67          
(0, 1, 2)            38.04          
(0, 1, 3)            38.04          
(0, 1, 4)            38.04          
(0, 1, 5)            38.04          
(0, 2, 0)            9.34    